# Подготовка датасета и обучение YOLO для детекции цифр

1. Структура датасета
Организуйте данные в следующем формате:

Copy
dataset/
├── images/
│   ├── 0_001.png
│   ├── 0_002.png
│   ├── 1_001.png
│   └── ...
└── labels/
    ├── 0_001.txt
    ├── 0_002.txt
    ├── 1_001.txt
    └── ...

2. Разметка данных (YOLO формат)
Для каждого изображения создайте .txt файл с аннотациями в формате:

Copy
<class_id> <x_center> <y_center> <width> <height>

где координаты нормированы на [0, 1] относительно размеров изображения.

Пример labels/0_001.txt:

0 0.5 0.5 0.2 0.3

3. Генерация разметки (автоматическая)
Если цифры занимают все изображение, используйте скрипт:

In [4]:
import os
import cv2

In [2]:
def generate_labels(dataset_root, output_dir):
    """
    Генерирует YOLO-разметку для изображений цифр, где каждый класс в отдельной папке.
    
    :param dataset_root: Путь к корневой папке датасета (содержит папки 0-9)
    :param output_dir: Папка для сохранения файлов разметки
    """
    # Создаем папки для изображений и разметки
    images_dir = os.path.join(output_dir, 'images')
    labels_dir = os.path.join(output_dir, 'labels')
    
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)
    
    # Проходим по всем папкам с цифрами (0-9)
    for class_dir in sorted(os.listdir(dataset_root)):
        class_path = os.path.join(dataset_root, class_dir)
        
        if not os.path.isdir(class_path):
            continue
            
        try:
            class_id = int(class_dir)  # Имя папки - это класс цифры
        except ValueError:
            continue  # Пропускаем нецифровые папки
        
        # Обрабатываем все PNG-файлы в папке класса
        for img_name in os.listdir(class_path):
            if not img_name.endswith('.png'):
                continue
                
            # Полный путь к исходному изображению
            src_img_path = os.path.join(class_path, img_name)
            
            # Новое имя файла (сохраняем структуру "цифра_номер.png")
            new_img_name = f"{class_id}_{img_name.split('_')[1]}"
            dest_img_path = os.path.join(images_dir, new_img_name)
            
            # Копируем изображение в единую папку
            img = cv2.imread(src_img_path)
            cv2.imwrite(dest_img_path, img)
            
            # Генерируем разметку YOLO (всё изображение - одна цифра)
            h, w = img.shape[:2]
            label_content = f"{class_id} 0.5 0.5 1.0 1.0"  # Центр, ширина и высота 100%
            
            # Сохраняем разметку
            label_name = new_img_name.replace('.png', '.txt')
            label_path = os.path.join(labels_dir, label_name)
            
            with open(label_path, 'w') as f:
                f.write(label_content)
    
    print(f"Обработка завершена. Изображения сохранены в {images_dir}, разметка в {labels_dir}")

In [3]:
generate_labels('/home/lastinm/PROJECTS/DATA/digits/images',            # Папка с подпапками 0-9
                 '/home/lastinm/PROJECTS/DATA/digits (yolo format)')    # Куда сохранить подготовленные данные

Обработка завершена. Изображения сохранены в /home/lastinm/PROJECTS/DATA/digits (yolo format)/images, разметка в /home/lastinm/PROJECTS/DATA/digits (yolo format)/labels


## Разделение на train/val

In [4]:
import random
from shutil import copyfile

In [5]:
images_dir = '/home/lastinm/PROJECTS/DATA/digits (yolo format)/images'
labels_dir = '/home/lastinm/PROJECTS/DATA/digits (yolo format)/labels'

In [6]:
# После генерации всех данных:
all_images = os.listdir(images_dir)
random.shuffle(all_images)

# 80% train, 20% val
split_idx = int(0.8 * len(all_images))
train_images = all_images[:split_idx]
val_images = all_images[split_idx:]

# Создаем подпапки
os.makedirs('/home/lastinm/PROJECTS/DATA/digits (yolo format)/train/images', exist_ok=True)
os.makedirs('/home/lastinm/PROJECTS/DATA/digits (yolo format)/train/labels', exist_ok=True)
os.makedirs('/home/lastinm/PROJECTS/DATA/digits (yolo format)/val/images', exist_ok=True)
os.makedirs('/home/lastinm/PROJECTS/DATA/digits (yolo format)/val/labels', exist_ok=True)

# Копируем файлы
for img_name in train_images:
    # Изображения
    src = os.path.join(images_dir, img_name)
    dst = os.path.join('/home/lastinm/PROJECTS/DATA/digits (yolo format)/train/images', img_name)
    copyfile(src, dst)
    
    # Разметка
    label_name = img_name.replace('.png', '.txt')
    src = os.path.join(labels_dir, label_name)
    dst = os.path.join('/home/lastinm/PROJECTS/DATA/digits (yolo format)/train/labels', label_name)
    copyfile(src, dst)

# Аналогично для val...
for img_name in val_images:
    # Изображения
    src = os.path.join(images_dir, img_name)
    dst = os.path.join('/home/lastinm/PROJECTS/DATA/digits (yolo format)/val/images', img_name)
    copyfile(src, dst)
    
    # Разметка
    label_name = img_name.replace('.png', '.txt')
    src = os.path.join(labels_dir, label_name)
    dst = os.path.join('/home/lastinm/PROJECTS/DATA/digits (yolo format)/val/labels', label_name)
    copyfile(src, dst)

# Обучение модели

In [ ]:
%pip install ultralytics

In [1]:
from ultralytics import YOLO

In [2]:
# Загрузка предобученной модели
#model = YOLO('yolov8s.pt')  # small-версия
model = YOLO('yolov8n.pt')  # nano-версия

# Обучение
results = model.train(
    data='/home/lastinm/PROJECTS/DATA/syntetic digits/data.yaml',
    epochs=150,
    imgsz=640,  # Модель сама сделает ресайз с сохранением пропорций
    batch=40,
    augment=True,  # Автоматическая аугментация
    # hsv_h=0.1,  # Цветовые искажения
    # hsv_s=0.7,
    # hsv_v=0.4,
    translate=0.2,  # Сдвиги
    scale=0.5,      # Масштабирование
    fliplr=0.5,     # Горизонтальное отражение
    mosaic=1.0,      # Мозаика (особенно полезно для мелких объектов)
    overlap_mask=True,  # Для плотных цифр
#    rect=True,     # Прямоугольное обучение (оптимизирует память)
    device='0'     # GPU
)

New https://pypi.org/project/ultralytics/8.3.113 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.111 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4070, 11875MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/lastinm/PROJECTS/DATA/syntetic digits/data.yaml, epochs=150, time=None, patience=100, batch=40, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, sh

train: Scanning /home/lastinm/PROJECTS/DATA/syntetic digits/train/labels.cache... 4000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4000/4000 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 3.0±3.4 ms, read: 35.4±12.4 MB/s, size: 11.4 KB)


val: Scanning /home/lastinm/PROJECTS/DATA/syntetic digits/val/labels.cache... 1000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1000/1000 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.000625), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train
Starting training for 150 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    150/150      5.64G     0.1032     0.1157     0.7646        117        640: 100%|██████████| 100/100 [00:13<00:00,  7.29it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:02<00:00,  6.48it/s]

                   all       1000       3025          1          1      0.995      0.995



150 epochs completed in 0.680 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.111 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4070, 11875MiB)
Model summary (fused): 72 layers, 3,007,598 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:04<00:00,  3.22it/s]


                   all       1000       3025          1          1      0.995      0.989
                     0        265        297          1          1      0.995       0.99
                     1        282        319          1          1      0.995      0.991
                     2        250        287      0.999          1      0.995      0.989
                     3        269        300          1          1      0.995      0.989
                     4        246        280          1          1      0.995       0.99
                     5        286        329      0.998          1      0.995      0.986
                     6        257        282          1          1      0.995      0.991
                     7        279        317          1          1      0.995       0.99
                     8        274        299          1          1      0.995      0.991
                     9        275        315          1          1      0.995      0.986
Speed: 0.1ms preproce

# Inference модели

In [3]:
from ultralytics import YOLO
import cv2
import supervision as sv

In [ ]:
image_path = "artefacts/2_0.93_AgACAgIAAxkBAAICb2gDc1odBGUeYrwLDCqbVKbzKukKAALx7jEb9jsYSCYoYdOUlI6-AQADAgADeAADNgQ.jpeg"

In [9]:
model = YOLO('/home/lastinm/PROJECTS/credit_cards_detection/notebooks/runs/detect/train3/weights/best.pt')
image = cv2.imread(image_path)

results = model(image, verbose=False)[0]
detections = sv.Detections.from_ultralytics(results).with_nms()

box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

annotated_image = image.copy()
annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections)
annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)

sv.plot_image(annotated_image)

<Figure size 1200x1200 with 1 Axes>